# GT Educação

O GT Educação enviou uma série de formulários com indicadores desejados para o painel. A maioria desses indicadores estão disponíveis no Portal de Dados Abertos da Prefeitura de São Paulo (http://dados.prefeitura.sp.gov.br/).

Portanto, vamos carregar o módulo de downloads do portal de dados abertos.

In [1]:
from core.downloads import dadosabertos as da

# Extração e transformação inicial

## Número de alunos da Rede Municipal de Ensino

O primeiro indicador é o número de alunos da rede municipal, segmentado por nível educacional (Creche, Pré Escola, Ensino Fundamental I, Ensino Fundamental II, EJA I, EJA II, Ensino Médio, Ed. Prof.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/demanda-e-matriculas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'demanda-e-matriculas'
resources = da.package_resources(pkg_name, '23')
resources

Sabendo o id do arquivo de dezembro de 2023, vamos carregar o recurso como um dataframe.

In [ ]:
resource_id = 'a63afccb-1a19-4438-96b1-7a13c4ccb586'
mat_23 = da.load_resource(resource_id, pandas_kwargs=dict(header=[0,1]))
mat_23

O dataframe contem dados de matrículas, matrículas em processo e demanda não atendida. Como desejamos apenas matrículas, vamos remover as outras colunas.

In [ ]:
mat_23 = mat_23[['Distrito', 'Matrículas']]
mat_23

Agora, sabendo que temos apenas dados sobre matrículas, vamos ajustar as colunas.

In [ ]:
mat_23 = (mat_23
 .droplevel(0, axis='columns')
 .rename(columns={'Unnamed: 0_level_1': 'Distrito'}))

mat_23

As últimas linhas do dataframe são apenas dados de totalização e notas, então vamos excluí-las também.

In [ ]:
mat_23 = mat_23.loc[mat_23['Distrito'].str.lower()!='total']
mat_23 = mat_23.loc[~mat_23.iloc[:,1].isna()]
mat_23

Finalmente, vamos "despivotar" a tabela, transformando o nome das colunas em uma nova coluna chamada "Nível educacional" e unificando as colunas de valor sob uma coluna chamada matrículas.

In [ ]:
mat_23 = mat_23.melt(
    id_vars='Distrito',
    var_name='Nível Educacional',
    value_name='Matrículas'
)
mat_23

## Número de professores da Rede Municipal de Ensino

O segundo indicador solicitado é o número de professores da Rede Municipal de Ensino, com recortes por Raça, Sexo, Nível de Ensino (Educação Infantil, Ensino Fundamental etc.), Tipo de Unidade de atuação (EMEF, EMEI etc.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/microdados-servidores-perfil, porém os arquivos possuem informação sobre todos os servidores da SME, o que inclui os servidores administrativos, que precisarão ser excluídos.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'microdados-servidores-perfil'
resources = da.package_resources(pkg_name, '23')
resources

Aparentemente, os dados de perfil dos servidores possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
perf_23 = da.load_resource(resource_id,
                           pandas_kwargs={'encoding': 'utf-8'})
perf_23

Com os dados baixados, vamos selecionar as colunas para atender às segmentações propostas no formulário.

In [ ]:
# Raça, Sexo, Nível de Ensino (Educação Infantil, Ensino Fundamental etc.), Tipo de Unidade de atuação (EMEF, EMEI etc.).

perf_cols = [
    'CD_SEXO',
    'CD_RACA_COR',
    'DC_RACA_COR',
    'CD_DEF',
    'CD_CARGO_BASE',
    'DC_CARGO_BASE',
    'CD_CARGO_ATUAL',
    'DC_CARGO_ATUAL',
    'CD_AREA_ATUACAO_ATUAL',
    'DC_AREA_ATUACAO_ATUAL',
    'CD_UNIDADE_ATUAL',
    'TP_UNIDADE_ATUAL',
    'DC_UNIDADE_ATUAL',
]

perf_23 = perf_23.loc[:, perf_cols]
perf_23

O formulário solicita também a exclusão de professores aposentados e servidores administrativos do conjunto de dados. Apesar de existir uma coluna com um valor "aposentado", a `ACUMULO_SITUACAO_SEGUNDO_VINCULO`, essa coluna parece se referir à situação do segundo vínculo, e não do vínculo com a PMSP. Além disso, a lista de servidores aposentados da Secretaria Municipal de Educação tem mais de 94 mil registros, então esses registros provavelmente não estão nos dados de perfil dos servidores.

Para excluir os servidores administrativos, vamos primeiro avaliar quais cargos estão presentes na base.

In [ ]:
df_cargos = perf_23[['CD_CARGO_ATUAL', 'DC_CARGO_ATUAL']].copy()
df_cargos = df_cargos.drop_duplicates().reset_index(drop=True)

df_cargos

Dado a quantidade de cargos, vamos fitrar a lista com base em uma regra geral, mantendo apenas os cargos que possual a string "PROF" em sua descrição.

In [ ]:
filtro_prof = df_cargos['DC_CARGO_ATUAL'].str.lower().str.contains('prof')
df_cargos_prof = df_cargos.loc[filtro_prof]

df_cargos_prof

Por último, filtramos a base geral com base nessa lista de cargos.

In [ ]:
perf_23 = (
    perf_23
    .loc[
        perf_23['CD_CARGO_ATUAL'].isin(df_cargos_prof['CD_CARGO_ATUAL'].tolist())
    ]
)

perf_23

## Número de Unidades Escolares

O terceiro indicador solicitado é o número de Unidades Escolares da Rede Municipal de Ensino, com recortes por Tipo de Escola (EMEI, EMEF etc.) e Distrito.

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/cadastro-de-escolas-municipais-conveniadas-e-privadas.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'cadastro-de-escolas-municipais-conveniadas-e-privadas'
resources = da.package_resources(pkg_name, '23')
resources

Aparentemente, os dados de unidades escolares possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
escolas_23 = da.load_resource(resource_id)
escolas_23

Os dados das escolas também serão usados com outros conjuntos de dados para regionalizá-los, então vamos manter também algumas colunas de identificação das escolas para facilitar esse uso o posterior. Vamos também filtrar apenas as escolas ativas.

In [ ]:
escolas_cols = ['CODESC',
                'TIPOESC',
                'NOMES',
                'SUBPREF',
                'DISTRITO',
                'CODINEP',
                'CD_CIE',
                'NOME_ANT',
                'REDE']

escolas_filtro = escolas_23['SITUACAO'].str.lower().str.contains('ativa')
escolas_23 = escolas_23.loc[escolas_filtro, escolas_cols]

escolas_23

## Perfil dos alunos

O quinto indicador solicitado é o número de alunos da Rede Municipal de Ensino, com recortes por Raça, Sexo, existência de Necessidades Especiais, Distrito, Rede de Ensino (Administração Direta ou Rede Conveniada/Parceira).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/perfil-dos-educandos-cor-raca-idade-sexo-necessidades-educacionais-especiais.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Para padronizar o período de todos os conjuntos, vamos focar em dezembro de 2023.

In [ ]:
pkg_name = 'perfil-dos-educandos-cor-raca-idade-sexo-necessidades-educacionais-especiais'
resources = da.package_resources(pkg_name, '23')
resources

Aparentemente, os dados de perfil dos educandos possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
alunos_23 = da.load_resource(resource_id)
alunos_23

In [ ]:
alunos_cols = ['DISTRITO',
               'REDE',
               'MODAL',
               'SEXO',
               'NEE',
               'RACA',
               'Qtde']

alunos_23 = alunos_23[alunos_cols]

alunos_23

## Número de alunos estrangeiros

O sétimo indicador é o número de alunos estrangeiros da rede municipal, segmentado por País de Procedência, Distrito, Tipo de Rede (Direta ou Conveniada/Parceira), Etapa (Creche, fundamental, EJA etc.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/educandos-estrangeiros-por-nacionalidade. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados.

In [ ]:
pkg_name = 'educandos-estrangeiros-por-nacionalidade'
resources = da.package_resources(pkg_name, '23')
resources

Os dados de alunos estrangeiros parecem estar disponíveis anualmente, mas no mês de junho. Como não existe arquivo para dezembro de 2023, vamos utilizar o arquivo de junho de 2024.

In [ ]:
pkg_name = 'educandos-estrangeiros-por-nacionalidade'
resources = da.package_resources(pkg_name, '24')
resources

In [ ]:
resource_id = resources[2]['id']
alunos_est_2024 = da.load_resource(resource_id)
alunos_est_2024

In [ ]:
# Recortes: País de Procedência, Distrito, Tipo de Rede (Direta ou Conveniada/Parceira), Etapa (Creche, fundamental, EJA etc.)
alunos_bf_cols = ['DISTRITO',
                   'COD_PAIS',
                   'NOME_PAIS',
                   'ETAPA',
                   'REDE',
                   'QTDE']

alunos_est_2024 = alunos_est_2024[alunos_bf_cols]

alunos_est_2024

## Número de alunos beneficiários do Bolsa Família

O oitavo indicador é o número de alunos beneficiários do Bolsa Família da rede municipal, segmentado por Distrito e nível de ensino.

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/beneficiarios-programa-bolsa-familia-creches-municipais-e-conveniadas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

As notas (que podem ser vistas na saída abaixo) desse conjunto de dados incluem a informação de que a estrutura do conjunto mudou a partir de 2024 e, por isso, foi criado um novo conjunto de dados com a estrutura mais recente, disponível em http://dados.prefeitura.sp.gov.br/dataset/beneficiarios-do-programa-bolsa-familia-por-etapa-de-ensino-da-rede-municipal-de-educacao. Vamos utilizar essa versão mais recente e os dados de julho de 2024, assim como o número de alunos estrangeiros.

In [ ]:
pkg_name = 'beneficiarios-programa-bolsa-familia-creches-municipais-e-conveniadas'
pkg = da.package_show(pkg_name)
print(pkg['notes'])

In [ ]:
pkg_name = 'beneficiarios-do-programa-bolsa-familia-por-etapa-de-ensino-da-rede-municipal-de-educacao'
resources = da.package_resources(pkg_name, '24')
resources

In [ ]:
resource_id = resources[2]['id']
alunos_bf_2024 = da.load_resource(resource_id)
alunos_bf_2024

In [ ]:
# Recortes: Alunos Beneficiários do Bolsa-Família por distrito, por nível de Ensino
alunos_bf_cols = ['CODEOL',
                   'TIPO',
                   'UNIDADE',
                   'DIRETORIA',
                   'DISTRITO',
                   'CRECHE',
                   'PRE ESCOLA',
                   'ENSINO FUNDAMENTAL',
                   'EDUCAÇÃO PROFISSIONAL',
                   'ENSINO MEDIO',
                   'EDUCAÇÃO ESPECIAL']

alunos_bf_2024 = alunos_bf_2024[alunos_bf_cols]

alunos_bf_2024

In [ ]:
alunos_bf_id_vars = alunos_bf_cols[0:5]

alunos_bf_2024 = alunos_bf_2024.melt(id_vars=alunos_bf_id_vars,
                    var_name='NIVEL',
                    value_name='QTDE')

alunos_bf_2024

# Transformação e mesclagem de dados

Para facilitar o trabalho dos dados no Qlik Sense, vamos avaliar como é a melhor forma de retrabalhar os dados de acordo com o paradigma fato-dimensão.

## Nível Escolar

A primeira dimensão que parece fazer sentido é o Nível escolar. Vamos avaliar como aparece em cada tabela.

In [ ]:
mat_23['Nível Educacional'].value_counts()

In [ ]:
perf_23[['CD_AREA_ATUACAO_ATUAL', 'DC_AREA_ATUACAO_ATUAL']].value_counts()

In [ ]:
alunos_23['MODAL'].value_counts()

In [ ]:
alunos_est_2024['ETAPA'].value_counts()

In [ ]:
alunos_bf_2024['NIVEL'].value_counts()